# Task
We want to achieve the following workflow:

1. do addition until reaching a certain value

**Note:**

**This might seem simple and general for a python program. But you can imagine, we can swap 'do addition' to something else, like machine learning training.**

**Run through this python notebook and think about the differences between the while loop and dflow recurse steps.**

In [1]:
from dflow import (
    InputParameter,
    Inputs,
    Step,
    Steps,
    Workflow,
)
from dflow.python import OP, OPIO, OPIOSign, PythonOPTemplate

Plus1 is a Python OP using python:3.8 image which prints the current number of iter and outputs the number of next iter.

- input: the number of current iter
- output: the number of next iter

In [2]:
class Plus1(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'iter': int
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'iter': int
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        print("This is iter %s" % op_in["iter"])
        return OPIO({
            'iter': op_in['iter'] + 1
        })

steps is a template of type Steps. It needs two parameters, iter whose default value is 0 and limit with value 3.

In [3]:
steps = Steps(
    name="iter",
    inputs=Inputs(
        parameters={"iter": InputParameter(value=0), "limit": InputParameter(value=3)}
    ),
)

**Note**

1. We are using `Steps`. 

2. `Steps` is essentially an `OPTemplate` (which is a kind of super OP, [See source code](https://github.com/deepmodeling/dflow/blob/master/src/dflow/steps.py)). Using this function, we can construct more complicated workflow. For instance, we are using `Steps` to create a while loop. 

The addition step:
- it uses Plus1 template
- its input parameter is from steps.inputs.parameters["iter"]

In [4]:
addition = Step(
    name="add", template=PythonOPTemplate(Plus1, image="python:3.8"), parameters={"iter": steps.inputs.parameters["iter"]}
)

the loop step: 
- update the number of iter by addition.outputs.parameters["iter"]
- the loop's condition is when the iter number of additon step less than the limit.

In [5]:
loop = Step(
    name="loop",
    template=steps,
    parameters={"iter": addition.outputs.parameters["iter"]},
    when=addition.outputs.parameters["iter"] < steps.inputs.parameters["limit"],
)

This step use steps as its template (note that Steps is a subclass of OPTemplate), meanwhile the steps it used contains this step, which gives a recursion. The recursion will stop when the "when" condition is not satisfied (after 3 loops in this example)

In [6]:
steps.add(addition)
steps.add(loop)

In [7]:
wf = Workflow("recurse", steps=steps)
wf.submit();

Workflow has been submitted (ID: recurse-d2w47, UID: 333bbd6f-575c-43ff-b2e8-402d4c1ec2be)
